In [26]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)



Vorüberlegung, um Moderatoren zu bestimmen:
Moderatoren sind Nutzer, die in einem Zeitraum von 5 Minuten mehr als einen Pixel setzen konnten.

D.h.
SELECT d1.user_id

FROM dataFrame d1 JOIN dataFrame d2 ON d1.user_id = d2.user_id

WHERE d1.t > d2.t - 60 * 5 (60*5 entspricht 5 Minuten)

Bzw. WHERE d1.t > d2.t - 60 * 4 + 58 (also 2 Sek. Toleranz)

Wie setzt man das in dataFrames um???

<div style="color:lightblue">
(Sebastian):
schau mal den self join an den ich gemacht habe in 0.1-heuer-pixelklassen

Beispiel aus meinem Code:
```
selfjoinedFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'),F.col('df1.user_id') == F.col('df2.user_id'),'inner')
selfjoinedFrame = selfjoinedFrame.where('df2.t > df1.t').where('df2.t <= df1.t + 2*'+str(RZModel.max_t_dist))
selfjoinedFrame = selfjoinedFrame.where('Abs(df2.x - df1.x) <= 2*'+str(RZModel.max_xy_dist))
selfjoinedFrame = selfjoinedFrame.where('Abs(df2.y - df1.y) <= 2*'+str(RZModel.max_xy_dist))
selfjoinedFrame = selfjoinedFrame.select(F.col('df1.user_id'),F.col('df1.x').alias("x1"),F.col('df2.x').alias('x2'),F.col('df1.y').alias("y1"),F.col('df2.y').alias('y2'),F.col('df1.t').alias("t1"),F.col('df2.t').alias('t2'),F.col('df1.pixel_color').alias("c1"),F.col('df2.pixel_color').alias('c2'))
```

So kannst du das auch machen:
* df2.t > df1.t , dann ist das schonmal geordnet und du brauchst keine doppler wegen Reihenfolge betrachten
* df2.t < df1.t + (60*5) - eventuell 2 Sekunden grace period geben. Also (60*4 + 58) Echte Nutzer können durch Serverlag vlt ein paar Sekunden früher setzen - Moderatoren sind hier deutliche Ausreißer!
* dann diesen DataFrame wieder pro Nutzer aggregieren um zu gucken wie viele Pixel ein Nutzer anomal gesetzt hat :)

</div>

In [36]:
import pyspark.sql.functions as F
from src.models.models import RZModel

selfjoinedFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'), F.col('df1.user_id') == F.col('df2.user_id'),'inner')
selfjoinedFrame = selfjoinedFrame.where('df2.t > df1.t').where('df2.t <= df1.t + 2*'+str(RZModel.max_t_dist))
selfjoinedFrame = selfjoinedFrame.where('Abs(df2.x - df1.x) <= 2*'+str(RZModel.max_xy_dist))
selfjoinedFrame = selfjoinedFrame.where('Abs(df2.y - df1.y) <= 2*'+str(RZModel.max_xy_dist))
selfjoinedFrame = selfjoinedFrame.select(F.col('df1.user_id'),F.col('df1.x').alias("x1"),F.col('df2.x').alias('x2'),F.col('df1.y').alias("y1"),F.col('df2.y').alias('y2'),F.col('df1.t').alias("t1"),F.col('df2.t').alias('t2'),F.col('df1.pixel_color').alias("c1"),F.col('df2.pixel_color').alias('c2'))
selfjoinedFrame.show()

+--------------------+---+---+----+----+----+----+-------+-------+
|             user_id| x1| x2|  y1|  y2|  t1|  t2|     c1|     c2|
+--------------------+---+---+----+----+----+----+-------+-------+
|++3l0k1HSXKiAsHkH...|684|668|1270|1211|1058|1381|#D4D7D9|#D4D7D9|
|++3l0k1HSXKiAsHkH...|684|664|1270|1240|1058|1721|#D4D7D9|#9C6926|
|++3l0k1HSXKiAsHkH...|684|668|1270|1211|1058|1381|#D4D7D9|#D4D7D9|
|++3l0k1HSXKiAsHkH...|684|664|1270|1240|1058|1721|#D4D7D9|#9C6926|
|++3l0k1HSXKiAsHkH...|684|667|1270|1240|1058|2057|#D4D7D9|#9C6926|
|++3l0k1HSXKiAsHkH...|684|663|1270|1240|1058|2399|#D4D7D9|#9C6926|
|++3l0k1HSXKiAsHkH...|684|653|1270|1229|1058|2779|#D4D7D9|#FFFFFF|
|++3l0k1HSXKiAsHkH...|684|615|1270|1237|1058|3106|#D4D7D9|#00A368|
|++3l0k1HSXKiAsHkH...|684|667|1270|1240|1058|2057|#D4D7D9|#9C6926|
|++3l0k1HSXKiAsHkH...|684|663|1270|1240|1058|2399|#D4D7D9|#9C6926|
|++3l0k1HSXKiAsHkH...|684|653|1270|1229|1058|2779|#D4D7D9|#FFFFFF|
|++3l0k1HSXKiAsHkH...|684|615|1270|1237|1058|3106|#D4D7D9|#00A

<div style="color:lightgreen">
Hmm... nicht das Ergebnis was ich brauche, hier sind die Zeitabstände alle LÄNGER als 5 Minuten...
</div>

In [37]:
selfjoinedFrame2 = dataFrame.alias('df1').join(dataFrame.alias('df2'), F.col('df1.user_id') == F.col('df2.user_id'),'inner')
selfjoinedFrame2 = selfjoinedFrame2.where('df1.t < df2.t').where('(df2.t - df1.t) < 60 * 4 + 58')
selfjoinedFrame2 = selfjoinedFrame2.select(F.col('df1.user_id'),F.col('df1.x').alias("x1"),F.col('df2.x').alias('x2'),F.col('df1.y').alias("y1"),F.col('df2.y').alias('y2'),F.col('df1.t').alias("t1"),F.col('df2.t').alias('t2'),F.col('df1.pixel_color').alias("c1"),F.col('df2.pixel_color').alias('c2'))
selfjoinedFrame2.show(5)

+-------+---+---+---+---+---+---+---+---+
|user_id| x1| x2| y1| y2| t1| t2| c1| c2|
+-------+---+---+---+---+---+---+---+---+
+-------+---+---+---+---+---+---+---+---+



<div style="color:lightgreen">
Heißt dass, dass wir nach dieser ersten Definition für Moderatoren KEINE Moderatoren in diesem Testdatensatz haben???
<div>

Bzw. eine zweite zu betrachtende Situation ist, dass Moderatoren auch mehrere Pixel auf einmal ändern können.

SELECT user_id, COUNT(x) --es reicht aus die x Koordinate zu zählen

FROM dataFrame

WHERE --Wie könnte die Where-Clause angegeben werden, dass nur ein bestimmter Zeitpunkt betrachtet wird?

HAVING COUNT(x) > 1

GROUP BY user_id;

<div style="color:lightblue">(Sebastian): Gruppieren nach t-Spalte und dann gucken wo count(t) > 1 ?</div>

In [29]:
from src.features.feature_functions import group_dataframe_by_time

timeGrouped = group_dataframe_by_time(dataFrame)
timeGrouped.printSchema()

root
 |-- t: long (nullable = true)
 |-- count: long (nullable = false)
 |-- collect_list(user_id): array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- collect_list(x): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(y): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)



Da es Probleme gibt, wenn die Spalte count einfach count heißt, muss diese zunächst umbenanntn werden, damit weitergearbeitet werden kann. (count wird nämlich als Funktion & nicht als Spalte des DataFrames angesehen)

In [30]:
timeGrouped = timeGrouped.withColumnRenamed('count','time_count')
timeGrouped.printSchema()

root
 |-- t: long (nullable = true)
 |-- time_count: long (nullable = false)
 |-- collect_list(user_id): array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- collect_list(x): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(y): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)



Damit wurde das DataFrame schon mal nach der Zeit gruppiert und entsprechend die Häufigkeit gezählt.
Wenn der Zeitpunkt nur einmal gezählt wurde, muss dieser rausgeschmießen werden, da ein Zeitpunkt mindestens 2 mal vorkommen muss, damit es zu einem von einem Moderator gesetzten Pixel gehören könnte.
Also nun die entsprechende bereinigung der Daten:

In [31]:
timeGrouped = timeGrouped.filter(timeGrouped.time_count > 1)
timeGrouped.show()

+---+----------+---------------------+--------------------+--------------------+-------------------------+
|  t|time_count|collect_list(user_id)|     collect_list(x)|     collect_list(y)|collect_list(pixel_color)|
+---+----------+---------------------+--------------------+--------------------+-------------------------+
|502|      1372| [fl3fslGUutRFVv5s...|[1803, 1307, 1073...|[828, 537, 180, 1...|     [#000000, #FFFFFF...|
|503|      1352| [KhKhrQXoREif0iKG...|[453, 1813, 1812,...|[1216, 820, 814, ...|     [#000000, #FFFFFF...|
|506|      1352| [RtdCr10DU/ryVoyP...|[862, 1794, 1580,...|[807, 743, 1500, ...|     [#FF4500, #51E9F4...|
|518|      1336| [0ri9DdoyY1cUGgPO...|[1813, 539, 918, ...|[810, 1240, 575, ...|     [#FFFFFF, #DE107F...|
|510|      1336| [SuvkvnbkdYwMBLGl...|[1538, 1146, 1812...|[507, 551, 815, 1...|     [#D4D7D9, #FFD635...|
|665|      1328| [/Z19GH2yLv1t/Rwn...|[285, 121, 1818, ...|[1669, 1819, 842,...|     [#515252, #FFB470...|
|800|      1328| [vLy9enKCuqE5Lfz5...

Ist es nicht besser direkt nach Nutzer und Zeit zu gruppieren? Dann hat man für jeden Nutzer direkt angezeigt, wie oft zu einem Zeitpunkt Pixel platziert wurden?
Dann wäre auch die Bereinigung der Daten effektiver als oben.

Hier der Versuch:

In [27]:
from src.features.feature_functions import group_dataframe_by_time_and_user

timeUserGrouped = group_dataframe_by_time_and_user(dataFrame)
#timeUserGrouped = group_dataframe_by_columns(dataFrame, ['t', 'user_id'])
timeUserGrouped.printSchema()

root
 |-- t: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- count: long (nullable = false)
 |-- collect_list(x): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(y): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [28]:
timeUserGrouped = timeUserGrouped.withColumnRenamed('count','time_count')

timeUserGrouped = timeUserGrouped.filter(timeUserGrouped.time_count > 1)
timeUserGrouped.show()

+----+--------------------+----------+--------------------+--------------------+-------------------------+
|   t|             user_id|time_count|     collect_list(x)|     collect_list(y)|collect_list(pixel_color)|
+----+--------------------+----------+--------------------+--------------------+-------------------------+
|2974|X8gpDfO6+Z87SKKTy...|         8|[1685, 904, 932, ...|[228, 1865, 1850,...|     [#FFD635, #000000...|
|2344|AxBTmhBMbr1sbDw/2...|         6|[1403, 777, 1741,...|[2, 8, 10, 2, 8, 10]|     [#FFA800, #FFA800...|
|1200|DIECtVqoJqBlL6A0C...|         6|[932, 920, 933, 9...|[1757, 1832, 1838...|     [#FFFFFF, #BE0039...|
|1726|AxBTmhBMbr1sbDw/2...|         6|[655, 612, 1575, ...|[1896, 1917, 7, 1...|     [#B44AC0, #E4ABFF...|
| 275|Qafq4ohzHG2/BRgOZ...|         4|[1757, 948, 1757,...|[919, 1877, 919, ...|     [#000000, #FFA800...|
|2789|UYHFPHnWs1F2WibcT...|         4|[1203, 1203, 1203...|[1843, 1843, 1843...|     [#FFFFFF, #FFFFFF...|
|2167|FqWlYosC7Oca6Vvqp...|         4

Das ist so noch nicht ganz korrekt...oder? Als ob es so viele Moderatoren gibt, abgesehen von den Dopplungen...

Manche User tauchen mehrmals auf, d.h. ich muss Dopplungen rausfiltern.

In [29]:
#timeUserGrouped = timeUserGrouped.dropDuplicates(['user_id'])
#timeUserGrouped.show()

In [31]:
row = dataFrame.select('user_id').distinct().count()
print(f'Anzahl der herkömmlichen Zeilen: {row}')

row_sel = timeUserGrouped.select('user_id').distinct().count()
print(f'Anzahl der angeblichen Moderatoren: {row_sel}')

Anzahl der herkömmlichen Zeilen: 733458


22/06/05 14:23:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 14:23:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 14:23:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 14:23:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 14:23:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 14:23:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 14:23:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 14:23:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


Anzahl der angeblichen Moderatoren: 733458
